In [0]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [0]:
%cp ./weibo_data_split.zip /content

In [0]:
!unzip weibo_data_split.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/weibo_data_split/train/real/real1118.jpg  
  inflating: /content/weibo_data_split/train/real/real1119.jpg  
  inflating: /content/weibo_data_split/train/real/real1120.jpg  
  inflating: /content/weibo_data_split/train/real/real1129.jpg  
  inflating: /content/weibo_data_split/train/real/real1130.jpg  
  inflating: /content/weibo_data_split/train/real/real1131.jpg  
  inflating: /content/weibo_data_split/train/real/real1137.jpg  
  inflating: /content/weibo_data_split/train/real/real1146.jpg  
  inflating: /content/weibo_data_split/train/real/real1150.jpg  
  inflating: /content/weibo_data_split/train/real/real1157.jpg  
  inflating: /content/weibo_data_split/train/real/real1166.jpg  
  inflating: /content/weibo_data_split/train/real/real1167.jpg  
  inflating: /content/weibo_data_split/train/real/real1169.jpg  
  inflating: /content/weibo_data_split/train/real/real1170.jpg  
  inflating: /content/weibo_data_split/

In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import os

Using TensorFlow backend.


In [0]:
import numpy as np

In [0]:
# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = '/content/drive/My Drive/Colab Notebooks/bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

In [0]:
base_dir = '/content/weibo_data_split'
train_data_dir = base_dir +'/train'
validation_data_dir = base_dir +'/validation'

nb_train_samples = 2*len(os.listdir(train_data_dir+'/fake'))
nb_validation_samples = 2*len(os.listdir(validation_data_dir+'/fake'))
epochs = 50
batch_size = 64

In [0]:
# build the VGG16 network
model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')





58892288/58889256 [==============================] - 2s 0us/step






Model loaded.


In [0]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

In [0]:
new_model = Sequential()
new_model.add(model)
new_model.add(top_model)


In [0]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in new_model.layers[:25]:
    layer.trainable = False

In [0]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
new_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-3, momentum = 0.9),
              metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 6794 images belonging to 2 classes.
Found 970 images belonging to 2 classes.


In [0]:
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/cp_traLearn.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
from keras.callbacks import ModelCheckpoint
#callback
cp_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)

In [0]:
# fine-tune the model
history = new_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)


Epoch 1/50
106/106 [==============================] - 86s 810ms/step - loss: 5.3360 - acc: 0.6416 - val_loss: 5.2255 - val_acc: 0.6417
Epoch 2/50
106/106 [==============================] - 82s 771ms/step - loss: 5.3454 - acc: 0.6412 - val_loss: 5.1182 - val_acc: 0.6501
Epoch 3/50
106/106 [==============================] - 81s 764ms/step - loss: 5.3285 - acc: 0.6402 - val_loss: 5.4124 - val_acc: 0.6280
Epoch 4/50
106/106 [==============================] - 79s 750ms/step - loss: 5.3620 - acc: 0.6399 - val_loss: 5.1060 - val_acc: 0.6534
Epoch 5/50
106/106 [==============================] - 78s 737ms/step - loss: 5.2736 - acc: 0.6385 - val_loss: 5.5075 - val_acc: 0.6225
Epoch 6/50
106/106 [==============================] - 77s 729ms/step - loss: 5.3168 - acc: 0.6431 - val_loss: 5.0514 - val_acc: 0.6512
Epoch 7/50
106/106 [==============================] - 77s 725ms/step - loss: 5.3407 - acc: 0.6408 - val_loss: 4.9452 - val_acc: 0.6611
Epoch 8/50
106/106 [==============================] - 

In [0]:
new_model_json = new_model.to_json()
with open("new_model.json", "w") as json_file:
  json_file.write(new_model_json)
new_model.save_weights('new_model.h5')

In [0]:
Y_pred = new_model.evaluate_generator(validation_generator, nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred)
validation_generator.classes
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

Accuracy metric:

In [0]:
Y_pred

[5.221139512602816, 0.6422680414829057]

In [0]:
Y_pred = new_model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
y_pred = [1 if x >= 0.5 else 0 for x in Y_pred ]
validation_generator.classes
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))


Confusion Matrix
[[351 134]
 [341 144]]


In [0]:
print(classification_report(validation_generator.classes, y_pred))

              precision    recall  f1-score   support

           0       0.51      0.72      0.60       485
           1       0.52      0.30      0.38       485

    accuracy                           0.51       970
   macro avg       0.51      0.51      0.49       970
weighted avg       0.51      0.51      0.49       970

